In [1]:
!pip install hyperopt

   ---------------------------------------- 1.6/1.6 MB 14.4 MB/s eta 0:00:00
   ---------------------------------------- 200.5/200.5 kB 4.1 MB/s eta 0:00:00


In [18]:
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pandas as pd
import os


In [21]:
data = '../artifact/process_clean.csv'
# artifact\process_clean.csv

In [22]:
data = pd.read_csv(data)
data.head()

,Unnamed: 0,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,0,1,2,1,6,142,4,3,3,3,...,3,48,1,1,1,1,0,1,1,1
1,1,0,29,3,6,770,2,0,1,0,...,3,3,1,1,0,1,0,0,1,0
2,2,3,8,1,2,390,2,1,1,3,...,3,30,1,1,0,3,1,0,1,1
3,3,1,26,3,3,848,2,1,1,3,...,1,26,1,0,0,1,1,0,1,1
4,4,1,17,2,4,734,2,0,2,3,...,2,34,1,0,1,1,1,0,1,0


In [24]:
# Select features and target variable
x = data[data.columns.difference(['class'])]
y = data['class']

In [25]:
from sklearn.preprocessing import StandardScaler
scaler =  StandardScaler()
Xscale  = scaler.fit_transform(x)
Xscale

array([[-1.73031962,  2.78850999, -0.4645937 , ...,  0.62648137,
         1.04698668,  1.66990103],
       [-1.72685552, -1.19773321, -1.26286467, ...,  0.62648137,
        -0.76597727, -0.13053135],
       [-1.72339142,  1.19401271,  1.13194823, ..., -1.02650377,
         0.14050471, -0.13053135],
       ...,
       [ 1.72339142,  0.21959771,  1.13194823, ...,  0.62648137,
         1.04698668, -0.13053135],
       [ 1.72685552, -1.10915003, -0.4645937 , ...,  0.62648137,
         1.04698668, -0.13053135],
       [ 1.73031962, -0.7548173 , -1.26286467, ...,  1.86622023,
         1.04698668, -1.93096373]])

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xscale, y, test_size = 0.20, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((800, 21), (200, 21), (800,), (200,))

In [35]:
!pip install xgboost

In [49]:
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, \
                            precision_score, recall_score, f1_score, roc_auc_score, \
                            roc_curve,confusion_matrix

In [50]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180
    }

In [51]:
def hyperparameter_tuning(space):
    model = xgb.XGBClassifier(n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                         reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                         colsample_bytree=int(space['colsample_bytree']))
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    model.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="rmse",
            early_stopping_rounds=10,verbose=False)

    pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    #change the metric if you like
    return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [52]:
trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

SCORE:                                                                                                                 
0.705                                                                                                                  
SCORE:                                                                                                                 
0.295                                                                                                                  
SCORE:                                                                                                                 
0.295                                                                                                                  
  3%|█▊                                                          | 3/100 [00:00<00:13,  7.30trial/s, best loss: -0.705]

C:\Users\sudwa\anaconda3\Lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\sudwa\anaconda3\Lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\sudwa\anaconda3\Lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\sudwa\anaconda3\Lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warn

SCORE:                                                                                                                 
0.705                                                                                                                  
SCORE:                                                                                                                 
0.705                                                                                                                  
  5%|███                                                         | 5/100 [00:00<00:15,  5.98trial/s, best loss: -0.705]

C:\Users\sudwa\anaconda3\Lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\sudwa\anaconda3\Lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\sudwa\anaconda3\Lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\sudwa\anaconda3\Lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warn

SCORE:                                                                                                                 
0.705                                                                                                                  
SCORE:                                                                                                                 
0.705                                                                                                                  
  7%|████▏                                                       | 7/100 [00:01<00:15,  6.16trial/s, best loss: -0.705]

...

C:\Users\sudwa\anaconda3\Lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\sudwa\anaconda3\Lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\sudwa\anaconda3\Lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\sudwa\anaconda3\Lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warn

SCORE:                                                                                                                 
0.705                                                                                                                  
SCORE:                                                                                                                 
0.705                                                                                                                  
 99%|██████████████████████████████████████████████████████████▍| 99/100 [00:28<00:00,  4.03trial/s, best loss: -0.705]

C:\Users\sudwa\anaconda3\Lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\sudwa\anaconda3\Lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
0.705                                                                                                                  
100%|██████████████████████████████████████████████████████████| 100/100 [00:28<00:00,  3.52trial/s, best loss: -0.705]


C:\Users\sudwa\anaconda3\Lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\sudwa\anaconda3\Lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



In [53]:
print(best)

{'colsample_bytree': 0.773087472194852, 'gamma': 4.29391787111393, 'max_depth': 9.0, 'min_child_weight': 2.0, 'reg_alpha': 81.0, 'reg_lambda': 0.08704259553205762}
